In [1]:
import $ivy.`org.apache.spark::spark-sql:3.5.5`
import $ivy.`org.postgresql:postgresql:42.7.2`
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import $ivy.$
import $ivy.$
import org.apache.log4j.{Level, Logger}

In [2]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions._

import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions._

Создаём SparkSession.

In [3]:
val spark = SparkSession
                .builder()
                .master("local[*]")
                .appName("JDBC Data Source")
                .config("spark.driver.memory", "8g")
                .config("spark.log.level", "WARN")
                .getOrCreate()

import spark.implicits._

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/03/13 16:48:13 INFO SparkContext: Running Spark version 3.5.5
25/03/13 16:48:13 INFO SparkContext: OS info Mac OS X, 15.3.2, aarch64
25/03/13 16:48:13 INFO SparkContext: Java version 11.0.26
25/03/13 16:48:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting Spark log level to "WARN".


spark: SparkSession = org.apache.spark.sql.SparkSession@56fdc2be
import spark.implicits._

Задаём свойства подключения.

In [4]:
val driver = "org.postgresql.Driver"
val url = "jdbc:postgresql://localhost:5432/spark"
val user = "postgres"
val password = "postgres"

driver: String = "org.postgresql.Driver"
url: String = "jdbc:postgresql://localhost:5432/spark"
user: String = "postgres"
password: String = "postgres"

## Чтение таблицы целиком

### Вариант 1

In [5]:
val employees_df = spark.read
    .format("jdbc")
    .option("driver", driver)
    .option("url", url)
    .option("user", user)
    .option("password", password)
    .option("dbtable", "public.employees")
    .load()

employees_df.count()

employees_df: org.apache.spark.sql.package.DataFrame = [emp_no: int, birth_date: date ... 4 more fields]
res5_1: Long = 30003L

In [6]:
employees_df.printSchema()

root
 |-- emp_no: integer (nullable = true)
 |-- birth_date: date (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- hire_date: date (nullable = true)



In [7]:
employees_df.show(10)

+------+----------+----------+-----------+------+----------+
|emp_no|birth_date|first_name|  last_name|gender| hire_date|
+------+----------+----------+-----------+------+----------+
| 10010|1963-06-01| Duangkaew|   Piveteau|     F|1989-08-24|
| 10020|1952-12-24|    Mayuko|    Warwick|     M|1991-01-26|
| 10030|1958-07-14|     Elvis|    Demeyer|     M|1994-02-17|
| 10040|1959-09-13|     Weiyi|    Meriste|     F|1993-02-14|
| 10050|1958-05-21|   Yinghua|     Dredge|     M|1990-12-25|
| 10060|1961-10-15|  Breannda|Billingsley|     M|1987-11-02|
| 10070|1955-08-20|    Reuven| Garigliano|     M|1985-10-14|
| 10080|1957-12-03|    Premal|       Baek|     M|1985-11-19|
| 10090|1961-05-30|    Kendra|    Hofting|     M|1986-03-14|
| 10100|1953-04-21|  Hironobu|  Haraldson|     F|1987-09-21|
+------+----------+----------+-----------+------+----------+
only showing top 10 rows



### Вариант 2

In [8]:
import java.util.Properties

val connectionProperties = new Properties()
connectionProperties.put("user", user)
connectionProperties.put("password", password)

import java.util.Properties
connectionProperties: Properties = {password=postgres, user=postgres}
res8_2: Object = null
res8_3: Object = null

In [9]:
val df = spark.read.jdbc(url, "public.employees", connectionProperties)
df.count

df: org.apache.spark.sql.package.DataFrame = [emp_no: int, birth_date: date ... 4 more fields]
res9_1: Long = 30003L

In [10]:
df.printSchema()

root
 |-- emp_no: integer (nullable = true)
 |-- birth_date: date (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- hire_date: date (nullable = true)



In [11]:
df.show(10)

+------+----------+----------+-----------+------+----------+
|emp_no|birth_date|first_name|  last_name|gender| hire_date|
+------+----------+----------+-----------+------+----------+
| 10010|1963-06-01| Duangkaew|   Piveteau|     F|1989-08-24|
| 10020|1952-12-24|    Mayuko|    Warwick|     M|1991-01-26|
| 10030|1958-07-14|     Elvis|    Demeyer|     M|1994-02-17|
| 10040|1959-09-13|     Weiyi|    Meriste|     F|1993-02-14|
| 10050|1958-05-21|   Yinghua|     Dredge|     M|1990-12-25|
| 10060|1961-10-15|  Breannda|Billingsley|     M|1987-11-02|
| 10070|1955-08-20|    Reuven| Garigliano|     M|1985-10-14|
| 10080|1957-12-03|    Premal|       Baek|     M|1985-11-19|
| 10090|1961-05-30|    Kendra|    Hofting|     M|1986-03-14|
| 10100|1953-04-21|  Hironobu|  Haraldson|     F|1987-09-21|
+------+----------+----------+-----------+------+----------+
only showing top 10 rows



Проверим количество партиций.

In [12]:
df.rdd.getNumPartitions

res12: Int = 1

## Как распараллелить чтение?

### Партиционирование по столбцам

Добавим количество партиций к параметрам чтения таблицы.

In [13]:
val df101 = spark.read
    .format("jdbc")
    .option("driver", driver)
    .option("url", url)
    .option("user", user)
    .option("password", password)
    .option("dbtable", "public.employees")
    .option("numPartitions", "10")
    .load()

println(s"count = ${df101.count}")
println(s"num partitions = ${df101.rdd.getNumPartitions}")

count = 30003
num partitions = 1


df101: org.apache.spark.sql.package.DataFrame = [emp_no: int, birth_date: date ... 4 more fields]

Количество партиций не изменилось.

Узнаем минимальное и максимальное значения столбца *emp_no*

In [14]:
df.agg(min(col("emp_no")), max(col("emp_no"))).show()

+-----------+-----------+
|min(emp_no)|max(emp_no)|
+-----------+-----------+
|      10010|     499990|
+-----------+-----------+



In [15]:
val min_emp_no = df.agg(min(col("emp_no"))).collect()(0)(0).toString
val max_emp_no = df.agg(max(col("emp_no"))).collect()(0)(0).toString

println(s"min = $min_emp_no\nmax = $max_emp_no")

min = 10010
max = 499990


min_emp_no: String = "10010"
max_emp_no: String = "499990"

In [16]:
val df102 = spark.read
    .format("jdbc")
    .option("driver", driver)
    .option("url", url)
    .option("user", user)
    .option("password", password)
    .option("dbtable", "public.employees")
    .option("partitionColumn", "emp_no")
    .option("lowerBound", min_emp_no)
    .option("upperBound", max_emp_no)
    .option("numPartitions", "10")
    .load()

df102: org.apache.spark.sql.package.DataFrame = [emp_no: int, birth_date: date ... 4 more fields]

Number of partitions: 10

WHERE clauses of these partitions: 
- "emp_no" < 59008 or "emp_no" is null
- "emp_no" >= 59008 AND "emp_no" < 108006
- "emp_no" >= 108006 AND "emp_no" < 157004
- "emp_no" >= 157004 AND "emp_no" < 206002
- "emp_no" >= 206002 AND "emp_no" < 255000
- "emp_no" >= 255000 AND "emp_no" < 303998
- "emp_no" >= 303998 AND "emp_no" < 352996
- "emp_no" >= 352996 AND "emp_no" < 401994
- "emp_no" >= 401994 AND "emp_no" < 450992
- "emp_no" >= 450992

In [17]:
println(s"count = ${df102.count()}\nnum partitions = ${df102.rdd.getNumPartitions}")

count = 30003
num partitions = 10


In [18]:
df102.show(10)

+------+----------+----------+-----------+------+----------+
|emp_no|birth_date|first_name|  last_name|gender| hire_date|
+------+----------+----------+-----------+------+----------+
| 10010|1963-06-01| Duangkaew|   Piveteau|     F|1989-08-24|
| 10020|1952-12-24|    Mayuko|    Warwick|     M|1991-01-26|
| 10030|1958-07-14|     Elvis|    Demeyer|     M|1994-02-17|
| 10040|1959-09-13|     Weiyi|    Meriste|     F|1993-02-14|
| 10050|1958-05-21|   Yinghua|     Dredge|     M|1990-12-25|
| 10060|1961-10-15|  Breannda|Billingsley|     M|1987-11-02|
| 10070|1955-08-20|    Reuven| Garigliano|     M|1985-10-14|
| 10080|1957-12-03|    Premal|       Baek|     M|1985-11-19|
| 10090|1961-05-30|    Kendra|    Hofting|     M|1986-03-14|
| 10100|1953-04-21|  Hironobu|  Haraldson|     F|1987-09-21|
+------+----------+----------+-----------+------+----------+
only showing top 10 rows



Посмотрим сколько записей попало в каждую партицию

In [19]:
df102.rdd.foreachPartition(p => println(s"Partition count = ${p.length}"))

Partition count = 0
Partition count = 203
Partition count = 200
Partition count = 601
Partition count = 4500
Partition count = 4900
Partition count = 4900
Partition count = 4899
Partition count = 4900
Partition count = 4900


Это также можно получить другим способом

In [20]:
df102.rdd.mapPartitionsWithIndex{(p,i) => List((p, i.length)).iterator}.collect()

res20: Array[(Int, Int)] = Array(
  (0, 4900),
  (1, 4900),
  (2, 203),
  (3, 601),
  (4, 4899),
  (5, 4500),
  (6, 0),
  (7, 200),
  (8, 4900),
  (9, 4900)
)

Зададим в качестве *lowerBound* и *upperBound* произвольные значения (не min и max)

In [21]:
val df103 = spark.read
    .format("jdbc")
    .option("driver", driver)
    .option("url", url)
    .option("user", user)
    .option("password", password)
    .option("dbtable", "public.employees")
    .option("partitionColumn", "emp_no")
    .option("lowerBound", "20000")
    .option("upperBound", "50000")
    .option("numPartitions", "10")
    .load()

df103: org.apache.spark.sql.package.DataFrame = [emp_no: int, birth_date: date ... 4 more fields]

Number of partitions: 10

WHERE clauses of these partitions:
- "emp_no" < 23000 or "emp_no" is null
- "emp_no" >= 23000 AND "emp_no" < 26000
- "emp_no" >= 26000 AND "emp_no" < 29000
- "emp_no" >= 29000 AND "emp_no" < 32000
- "emp_no" >= 32000 AND "emp_no" < 35000
- "emp_no" >= 35000 AND "emp_no" < 38000
- "emp_no" >= 38000 AND "emp_no" < 41000
- "emp_no" >= 41000 AND "emp_no" < 44000
- "emp_no" >= 44000 AND "emp_no" < 47000
- "emp_no" >= 47000

Посмотрим сколько теперь записей попало в каждую партицию

In [22]:
println(s"count = ${df103.count()}\nnum partitions = ${df103.rdd.getNumPartitions}")

count = 30003
num partitions = 10


In [23]:
df103.rdd.mapPartitionsWithIndex{(p,i) => List((p, i.length)).iterator}.collect()

res23: Array[(Int, Int)] = Array(
  (0, 1299),
  (1, 300),
  (2, 300),
  (3, 300),
  (4, 300),
  (5, 300),
  (6, 300),
  (7, 300),
  (8, 300),
  (9, 26304)
)

In [24]:
df103.show(10)

+------+----------+----------+-----------+------+----------+
|emp_no|birth_date|first_name|  last_name|gender| hire_date|
+------+----------+----------+-----------+------+----------+
| 10010|1963-06-01| Duangkaew|   Piveteau|     F|1989-08-24|
| 10020|1952-12-24|    Mayuko|    Warwick|     M|1991-01-26|
| 10030|1958-07-14|     Elvis|    Demeyer|     M|1994-02-17|
| 10040|1959-09-13|     Weiyi|    Meriste|     F|1993-02-14|
| 10050|1958-05-21|   Yinghua|     Dredge|     M|1990-12-25|
| 10060|1961-10-15|  Breannda|Billingsley|     M|1987-11-02|
| 10070|1955-08-20|    Reuven| Garigliano|     M|1985-10-14|
| 10080|1957-12-03|    Premal|       Baek|     M|1985-11-19|
| 10090|1961-05-30|    Kendra|    Hofting|     M|1986-03-14|
| 10100|1953-04-21|  Hironobu|  Haraldson|     F|1987-09-21|
+------+----------+----------+-----------+------+----------+
only showing top 10 rows



### Партиционирование по предикатам

### Пример 1

Опредилим **два** предиката по значению столбца *gender*

In [25]:
val predicates = Array("gender = 'M'", "gender = 'F'")

val df_pred = spark.read.jdbc(url, "public.employees", predicates, connectionProperties)

println(s"count = ${df_pred.count}")
println(s"num partitions = ${df_pred.rdd.getNumPartitions}")

count = 30003
num partitions = 2


predicates: Array[String] = Array("gender = 'M'", "gender = 'F'")
df_pred: org.apache.spark.sql.package.DataFrame = [emp_no: int, birth_date: date ... 4 more fields]

Опредилим **один** предиката по одному значению столбца *gender*

In [26]:
val predicates1 = Array("gender = 'F'")

val df_pred1 = spark.read.jdbc(url, "public.employees", predicates1, connectionProperties)

println(s"count = ${df_pred1.count}")
println(s"num partitions = ${df_pred1.rdd.getNumPartitions}")

count = 12091
num partitions = 1


predicates1: Array[String] = Array("gender = 'F'")
df_pred1: org.apache.spark.sql.package.DataFrame = [emp_no: int, birth_date: date ... 4 more fields]

Опредилим **три** предиката по значению столбца *gender*

In [27]:
val predicates3 = Array("gender = 'M'", "gender = 'F'", "gender = 'M'")

val df_pred3 = spark.read.jdbc(url, "public.employees", predicates3, connectionProperties)

println(s"count = ${df_pred3.count}")
println(s"num partitions = ${df_pred3.rdd.getNumPartitions}")

count = 47915
num partitions = 3


predicates3: Array[String] = Array(
  "gender = 'M'",
  "gender = 'F'",
  "gender = 'M'"
)
df_pred3: org.apache.spark.sql.package.DataFrame = [emp_no: int, birth_date: date ... 4 more fields]

Опредилим **четыре** предиката по значению столбца *gender*

In [28]:
val predicates4 = Array("gender = 'M'", "gender = 'F'", "gender = 'M'", "gender = 'F'")

val df_pred4 = spark.read.jdbc(url, "public.employees", predicates4, connectionProperties)

println(s"count = ${df_pred4.count}")
println(s"num partitions = ${df_pred4.rdd.getNumPartitions}")

count = 60006
num partitions = 4


predicates4: Array[String] = Array(
  "gender = 'M'",
  "gender = 'F'",
  "gender = 'M'",
  "gender = 'F'"
)
df_pred4: org.apache.spark.sql.package.DataFrame = [emp_no: int, birth_date: date ... 4 more fields]

Посмотрим сколько записей для каждого значения столбца *gender* было в исходной таблице

In [29]:
df.groupBy(col("gender")).agg(count(col("emp_no"))).show()

+------+-------------+
|gender|count(emp_no)|
+------+-------------+
|     F|        12091|
|     M|        17912|
+------+-------------+



Сравним с количеством записей при применении трёх и четырёх предикатов

In [30]:
df_pred3.groupBy(col("gender")).agg(count(col("emp_no"))).show()

+------+-------------+
|gender|count(emp_no)|
+------+-------------+
|     M|        35824|
|     F|        12091|
+------+-------------+



In [31]:
df_pred4.groupBy(col("gender")).agg(count(col("emp_no"))).show()

+------+-------------+
|gender|count(emp_no)|
+------+-------------+
|     M|        35824|
|     F|        24182|
+------+-------------+



### Пример 2

Определим **два** предиката по условиям на значения столбца *emp_no* 

In [32]:
val predicates2 = Array("emp_no > 20000 and emp_no <= 50000", "emp_no >= 50000 and emp_no <= 100000")

val df_pred2 = spark.read.jdbc(url, "public.employees", predicates2, connectionProperties)

println(s"count = ${df_pred2.count}")
println(s"num partitions = ${df_pred2.rdd.getNumPartitions}")

count = 8001
num partitions = 2


predicates2: Array[String] = Array(
  "emp_no > 20000 and emp_no <= 50000",
  "emp_no >= 50000 and emp_no <= 100000"
)
df_pred2: org.apache.spark.sql.package.DataFrame = [emp_no: int, birth_date: date ... 4 more fields]

In [33]:
df_pred2.show(10)

+------+----------+----------+-----------+------+----------+
|emp_no|birth_date|first_name|  last_name|gender| hire_date|
+------+----------+----------+-----------+------+----------+
| 20010|1961-01-26|    Saniya|     Veccia|     M|1997-06-16|
| 20020|1962-07-22|     Akeel|     Covnot|     F|1996-03-02|
| 20030|1962-05-09|    Nitsan|Hoppenstand|     F|1988-11-18|
| 20040|1962-04-16|   Youjian|    Vingron|     M|1987-01-21|
| 20050|1959-02-27|  Guoxiang|   Greibach|     F|1991-03-18|
| 20060|1954-07-18|    Chrisa|Attimonelli|     F|1985-10-16|
| 20070|1959-04-13|       Tse|    Bellone|     M|1992-07-08|
| 20080|1962-03-22|   Odoardo|  Heiserman|     F|1991-07-01|
| 20090|1957-05-20| Serenella|   Kaltofen|     F|1986-06-24|
| 20100|1965-01-18|       Utz|     Heuter|     F|1986-06-15|
+------+----------+----------+-----------+------+----------+
only showing top 10 rows



Определим **один** предикат по условию на значения столбца *emp_no* 

In [34]:
val predicates22 = Array("emp_no > 20000 and emp_no <= 50000")

val df_pred22 = spark.read.jdbc(url, "public.employees", predicates22, connectionProperties)

println(s"count = ${df_pred22.count}")
println(s"num partitions = ${df_pred22.rdd.getNumPartitions}")

count = 3000
num partitions = 1


predicates22: Array[String] = Array("emp_no > 20000 and emp_no <= 50000")
df_pred22: org.apache.spark.sql.package.DataFrame = [emp_no: int, birth_date: date ... 4 more fields]

## Фильтрация

Выполним запрос к базе на выборку значений из таблицы с условием

In [35]:
val q = """select * from public.employees where emp_no > 20000 and emp_no <= 50000"""

val dfq = spark.read
    .format("jdbc")
    .option("driver", driver)
    .option("url", url)
    .option("user", user)
    .option("password", password)
    .option("query", q)
    .load()

dfq.count()

q: String = "select * from public.employees where emp_no > 20000 and emp_no <= 50000"
dfq: org.apache.spark.sql.package.DataFrame = [emp_no: int, birth_date: date ... 4 more fields]
res35_2: Long = 3000L

In [36]:
dfq.show(10)

+------+----------+----------+-----------+------+----------+
|emp_no|birth_date|first_name|  last_name|gender| hire_date|
+------+----------+----------+-----------+------+----------+
| 20010|1961-01-26|    Saniya|     Veccia|     M|1997-06-16|
| 20020|1962-07-22|     Akeel|     Covnot|     F|1996-03-02|
| 20030|1962-05-09|    Nitsan|Hoppenstand|     F|1988-11-18|
| 20040|1962-04-16|   Youjian|    Vingron|     M|1987-01-21|
| 20050|1959-02-27|  Guoxiang|   Greibach|     F|1991-03-18|
| 20060|1954-07-18|    Chrisa|Attimonelli|     F|1985-10-16|
| 20070|1959-04-13|       Tse|    Bellone|     M|1992-07-08|
| 20080|1962-03-22|   Odoardo|  Heiserman|     F|1991-07-01|
| 20090|1957-05-20| Serenella|   Kaltofen|     F|1986-06-24|
| 20100|1965-01-18|       Utz|     Heuter|     F|1986-06-15|
+------+----------+----------+-----------+------+----------+
only showing top 10 rows



## Соединения в базе

Выполним запрос к базе на выборку значений из соединения таблиц

In [37]:
val qj = """select e.emp_no, birth_date, first_name, last_name, gender, hire_date, salary, from_date, to_date from employees e join salaries s on e.emp_no = s.emp_no"""

val dfj = spark.read
    .format("jdbc")
    .option("driver", driver)
    .option("url", url)
    .option("user", user)
    .option("password", password)
    .option("query", qj)
    .load()

dfj.count()

qj: String = "select e.emp_no, birth_date, first_name, last_name, gender, hire_date, salary, from_date, to_date from employees e join salaries s on e.emp_no = s.emp_no"
dfj: org.apache.spark.sql.package.DataFrame = [emp_no: int, birth_date: date ... 7 more fields]
res37_2: Long = 283827L

In [38]:
dfj.show()

+------+----------+----------+---------+------+----------+------+----------+----------+
|emp_no|birth_date|first_name|last_name|gender| hire_date|salary| from_date|   to_date|
+------+----------+----------+---------+------+----------+------+----------+----------+
| 10010|1963-06-01| Duangkaew| Piveteau|     F|1989-08-24| 72488|1996-11-24|1997-11-24|
| 10010|1963-06-01| Duangkaew| Piveteau|     F|1989-08-24| 74347|1997-11-24|1998-11-24|
| 10010|1963-06-01| Duangkaew| Piveteau|     F|1989-08-24| 75405|1998-11-24|1999-11-24|
| 10010|1963-06-01| Duangkaew| Piveteau|     F|1989-08-24| 78194|1999-11-24|2000-11-23|
| 10010|1963-06-01| Duangkaew| Piveteau|     F|1989-08-24| 79580|2000-11-23|2001-11-23|
| 10010|1963-06-01| Duangkaew| Piveteau|     F|1989-08-24| 80324|2001-11-23|9999-01-01|
| 10020|1952-12-24|    Mayuko|  Warwick|     M|1991-01-26| 40000|1997-12-30|1998-12-30|
| 10020|1952-12-24|    Mayuko|  Warwick|     M|1991-01-26| 40647|1998-12-30|1999-12-30|
| 10020|1952-12-24|    Mayuko|  

## Запись в таблицу

Посмотрим на таблицу *employees*

In [39]:
employees_df.show(10)

+------+----------+----------+-----------+------+----------+
|emp_no|birth_date|first_name|  last_name|gender| hire_date|
+------+----------+----------+-----------+------+----------+
| 10010|1963-06-01| Duangkaew|   Piveteau|     F|1989-08-24|
| 10020|1952-12-24|    Mayuko|    Warwick|     M|1991-01-26|
| 10030|1958-07-14|     Elvis|    Demeyer|     M|1994-02-17|
| 10040|1959-09-13|     Weiyi|    Meriste|     F|1993-02-14|
| 10050|1958-05-21|   Yinghua|     Dredge|     M|1990-12-25|
| 10060|1961-10-15|  Breannda|Billingsley|     M|1987-11-02|
| 10070|1955-08-20|    Reuven| Garigliano|     M|1985-10-14|
| 10080|1957-12-03|    Premal|       Baek|     M|1985-11-19|
| 10090|1961-05-30|    Kendra|    Hofting|     M|1986-03-14|
| 10100|1953-04-21|  Hironobu|  Haraldson|     F|1987-09-21|
+------+----------+----------+-----------+------+----------+
only showing top 10 rows



Загрузим таблицу *salaries*

In [40]:
val salaries_df = spark.read
    .format("jdbc")
    .option("driver", driver)
    .option("url", url)
    .option("user", user)
    .option("password", password)
    .option("dbtable", "public.salaries")
    .load()

salaries_df.count()

salaries_df: org.apache.spark.sql.package.DataFrame = [emp_no: int, salary: int ... 2 more fields]
res40_1: Long = 283827L

In [41]:
salaries_df.show(10)

+------+------+----------+----------+
|emp_no|salary| from_date|   to_date|
+------+------+----------+----------+
| 10010| 72488|1996-11-24|1997-11-24|
| 10010| 74347|1997-11-24|1998-11-24|
| 10010| 75405|1998-11-24|1999-11-24|
| 10010| 78194|1999-11-24|2000-11-23|
| 10010| 79580|2000-11-23|2001-11-23|
| 10010| 80324|2001-11-23|9999-01-01|
| 10020| 40000|1997-12-30|1998-12-30|
| 10020| 40647|1998-12-30|1999-12-30|
| 10020| 43800|1999-12-30|2000-12-29|
| 10020| 44927|2000-12-29|2001-12-29|
+------+------+----------+----------+
only showing top 10 rows



Сделаем группировку по колонке *emp_no* и найдём максимальное значение колонки *salary*

In [42]:
val salaries_grouped = salaries_df.groupBy(col("emp_no")).agg(max(col("salary")).alias("max_salary"))

salaries_grouped.show(10)

+------+----------+
|emp_no|max_salary|
+------+----------+
| 12940|     85425|
| 13840|     41453|
| 14450|     75524|
| 14570|     72506|
| 15790|     79009|
| 17420|     97003|
| 18800|     88342|
| 19530|     47864|
| 21220|     86177|
| 21700|     68115|
+------+----------+
only showing top 10 rows



salaries_grouped: org.apache.spark.sql.package.DataFrame = [emp_no: int, max_salary: int]

Создадим новый Dataframe как результат соединения *employees_df* и агрегированного *salaries_df*

In [43]:
val employees_salaries_df = employees_df.join(salaries_grouped, "emp_no")

employees_salaries_df.show()

+------+----------+----------+---------+------+----------+----------+
|emp_no|birth_date|first_name|last_name|gender| hire_date|max_salary|
+------+----------+----------+---------+------+----------+----------+
| 12940|1953-10-25|  Odinaldo|   Farrar|     F|1987-12-12|     85425|
| 13840|1954-11-13|     Remco|    Demke|     M|1992-06-09|     41453|
| 14450|1963-08-01|  Fumitaka|Prochazka|     F|1985-04-26|     75524|
| 14570|1963-07-26|    Chinho|     Bala|     F|1994-11-17|     72506|
| 15790|1960-03-20|     Kokou| Schnabel|     M|1991-04-03|     79009|
| 17420|1964-05-22|     Jinpo|Stamatiou|     F|1987-07-31|     97003|
| 18800|1962-05-21|    Baruch|  Rosiles|     F|1990-07-06|     88342|
| 19530|1953-12-26|     Barry|   Dratva|     M|1997-09-02|     47864|
| 21220|1956-05-11|      Mana|  Murtagh|     M|1991-10-29|     86177|
| 21700|1963-07-12|       Urs|  Plesums|     F|1992-03-07|     68115|
| 27760|1959-12-05|    Zhigen|  Schrift|     M|1991-04-03|     63106|
| 28170|1964-05-19| 

employees_salaries_df: org.apache.spark.sql.package.DataFrame = [emp_no: int, birth_date: date ... 5 more fields]

Сохраним новый Dataframe в таблицу в базе. Таблицы с таким именем в базе не было. Она будет создана.

In [44]:
employees_salaries_df.write
    .format("jdbc")
    .option("driver", driver)
    .option("url", url)
    .option("user", user)
    .option("password", password)
    .option("dbtable", "public.employees_salaries")
    .save()

Если таблица с таким именем существовала в базе, то при сохранении надо использовать режим *overwrite*

In [45]:
employees_salaries_df.write
    .mode("overwrite")
    .format("jdbc")
    .option("driver", driver)
    .option("url", url)
    .option("user", user)
    .option("password", password)
    .option("dbtable", "public.employees_salaries")
    .option("truncate", "true")
    .save()

In [46]:
spark.read
    .format("jdbc")
    .option("driver", driver)
    .option("url", url)
    .option("user", user)
    .option("password", password)
    .option("dbtable", "public.employees_salaries")
    .load()
    .count()

res46: Long = 30003L

Если использовать режим *append* содержимое Dataframe будет добавлено в таблицу

In [47]:
employees_salaries_df.write
    .mode("append")
    .format("jdbc")
    .option("driver", driver)
    .option("url", url)
    .option("user", user)
    .option("password", password)
    .option("dbtable", "public.employees_salaries")
    .save()

In [48]:
spark.read
    .format("jdbc")
    .option("driver", driver)
    .option("url", url)
    .option("user", user)
    .option("password", password)
    .option("dbtable", "public.employees_salaries")
    .load()
    .count()

res48: Long = 60006L